# Analysis of Chinook Musical Store
- The goal of this analysis is to answer some business questions crucial for
its going concern is based.

In [227]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
"""run_query()  takes a SQL query and returns a pandas dataframe."""
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q,conn)

In [228]:
"""run_command()  takes a SQL command/query and executes it"""
def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None #tells SQLite to autocommit any changes 
        conn.execute(c)

In [229]:
"""show_tables() calls run_query() and return a list of tables/views in db"""
def show_tables():
    q = "SELECT name, type FROM sqlite_master WHERE type IN ('table','view');"
    return run_query(q)

In [230]:
db_info =show_tables()
print(db_info)

                   name   type
0                 album  table
1                artist  table
2              customer  table
3              employee  table
4                 genre  table
5               invoice  table
6          invoice_line  table
7            media_type  table
8              playlist  table
9        playlist_track  table
10                track  table
11  customer_sales_info   view
12   customer_sales_avg   view
13        purchase_info   view


# TASK 1:
- We want to know the genre of music with the highest purchase
in the USA

In [231]:
q = "select SUM(quantity) from invoice_line"
result = run_query(q)
print(result)
q1 = (
     "select g.name AS genre_name, "
     "SUM(il.quantity) AS total_sales, "
     "SUM(il.quantity) / CAST(4757 AS FLOAT) AS pct_sales "
     "from customer as c "
     "left join invoice as i on c.customer_id = i.customer_id "
     "left join invoice_line as il on i.invoice_id = il.invoice_id "
     "left join track on il.track_id = track.track_id "
     "left join genre AS g on track.genre_id = g.genre_id "
     "where c.country = 'USA' "
     "group by genre_name "
     "order by total_sales desc"
)
     

run_query(q1)

   SUM(quantity)
0           4757


,genre_name,total_sales,pct_sales
0,Rock,561,0.117931
1,Alternative & Punk,130,0.027328
2,Metal,124,0.026067
3,R&B/Soul,53,0.011141
4,Blues,36,0.007568
5,Alternative,35,0.007358
6,Latin,22,0.004625
7,Pop,22,0.004625
8,Hip Hop/Rap,20,0.004204
9,Jazz,14,0.002943


# Recommendation
- These artists: Red Tone (Punk), Slim Jim Bites (Blues) & Meteor and the Girls (Pop) should be added to the store. 

# TASK 2:
- Finds the total dollar amount of sales assigned to each sales support agent
 within the company.


In [232]:

q1 = (
    "select c.support_rep_id AS Sales_staff_ID, SUM(i.total) AS total_sales, "
    "count(c.customer_id) AS number_of_sales "
    "from employee AS e "
    "inner join customer AS c "
    "on e.employee_id = c.support_rep_id "
    "inner join invoice AS i "
    "on c.customer_id = i.customer_id "
    "where e.title = 'Sales Support Agent' "
    "group by support_rep_id "
    "order by total_sales desc "
        )
results = run_query(q1)
print(results)


   Sales_staff_ID  total_sales  number_of_sales
0               3      1731.51              212
1               4      1584.00              214
2               5      1393.92              188


# Statements
- The sales rep with ID = 3 appears to be high performing!


In [233]:
run_command("""DROP VIEW IF EXISTS customer_sales_info""")
run_command("""DROP VIEW IF EXISTS customer_sales_avg""")
run_command("""DROP VIEW IF EXISTS purchase_info""")
q2 = ''' CREATE VIEW customer_sales_info AS
    select c.customer_id, c.country, i.total, il.quantity 
    from customer AS c 
    left join invoice AS i on c.customer_id = i.customer_id 
    left join invoice_line AS il 
    on i.invoice_id = il.invoice_id 
'''
run_command(q2)

In [234]:
q3 = ''' CREATE VIEW customer_sales_avg AS
    SELECT customer_id, country, AVG(total) avg_customer_sales
    FROM customer_sales_info
    GROUP BY customer_id
    '''
run_command(q3)

In [235]:
q4 = '''SELECT csi.country,
        COUNT(csi.customer_id) AS total_num_customers,
        SUM(csi.total) AS total_sales_value,
        csa.avg_customer_sales, SUM(csi.total)/COUNT(csi.customer_id) AS avg_order
        FROM customer_sales_info AS csi
        INNER JOIN customer_sales_avg AS csa
        ON csi.customer_id = csa.customer_id
        GROUP BY csi.country
        ORDER BY avg_order desc'''
run_query(q4)


,country,total_num_customers,total_sales_value,avg_customer_sales,avg_order
0,Ireland,116,1433.52,12.357931,12.357931
1,Czech Republic,276,3183.84,11.544923,11.535652
2,Australia,82,940.50,11.469512,11.469512
3,Spain,99,1076.13,10.870000,10.870000
4,Hungary,79,830.61,10.514051,10.514051
5,India,185,1887.93,9.295000,10.205027
6,Germany,338,3441.24,10.354865,10.181183
7,Canada,541,5489.55,8.519211,10.147043
8,France,393,3972.87,7.518649,10.109084
9,United Kingdom,248,2498.76,9.157500,10.075645


# Recommendations:
- The future lies with the trio of Ireland, Czech Republic and Australia. With the highest average order value and the number of customers in their low hundreds, there is ample room for growth in those countries.

# Task 3:
Categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
- Number of invoices
- Percentage of invoices

In [236]:
q5 = '''CREATE VIEW purchase_info AS
        SELECT i.invoice_id, i.total, il.unit_price, il.quantity,
        CASE
            WHEN i.total = il.unit_price*il.quantity THEN 1
            WHEN i.total != il.unit_price*il.quantity THEN 0
        END AS purchase_type
        FROM invoice AS i
        LEFT JOIN invoice_line as il
        ON i.invoice_id = il.invoice_id
        
        '''
run_command(q5)

In [237]:
q6 ='''
    SELECT SUM(purchase_type) as track_purchase,
            SUM(1-purchase_type) as album_purchase,
            SUM(purchase_type)/CAST(COUNT(*) AS FLOAT) as pct_track_purch,
            SUM(1-purchase_type)/CAST(COUNT(*) AS FLOAT) as pct_album_purch
    FROM purchase_info
''' 
run_query(q6)

,track_purchase,album_purchase,pct_track_purch,pct_album_purch
0,38,4719,0.007988,0.992012


# Comment:
- Majority of purchases by customers are album purchases, and significantly so. Therefore, Chinook store should continue to buy full albums from record companies.